## Optimizing `max_mean_distance` parameter of `FrameImageDistanceSceneDetector`

In [1]:
from datetime import datetime, timedelta

from video699.configuration import get_configuration
from video699.video.annotated import (
    AnnotatedSampledVideoScreenDetector,
    AnnotatedSampledVideoPageDetector,
    evaluate_event_detector,
    get_videos,
)
from video699.video.scene import FrameImageDistanceSceneDetector
from video699.quadrangle.rtree import RTreeDequeConvexQuadrangleTracker
from video699.event.screen import ScreenEventDetector


CONFIGURATION = get_configuration()['FrameImageDistanceSceneDetector']

In [2]:
def accuracy(max_mean_distance):
    CONFIGURATION['max_mean_distance'] = str(max_mean_distance)
    num_successes_total = 0
    num_trials_total = 0
    for annotated_video in get_videos().values():        
        convex_quadrangle_tracker = RTreeDequeConvexQuadrangleTracker(2)
        screen_detector = AnnotatedSampledVideoScreenDetector()
        page_detector = AnnotatedSampledVideoPageDetector()
        video = FrameImageDistanceSceneDetector(annotated_video)
        screen_event_detector = ScreenEventDetector(
            video,
            convex_quadrangle_tracker,
            screen_detector,
            page_detector
        )
        num_successes, num_trials = evaluate_event_detector(annotated_video, screen_event_detector)
        num_successes_total += num_successes
        num_trials_total += num_trials
    accuracy = 1.0 * num_successes_total / num_trials_total
    return accuracy

In [4]:
start_time = datetime.now()
num_iterations = 0
while True:
    limit = lower_limit + 0.5 * (upper_limit - lower_limit)
    value = accuracy(limit)
    if value < 1.0:
        best_limit = lower_limit
        upper_limit = limit
    else:
        best_limit = limit
        lower_limit = limit
    num_iterations += 1
    if datetime.now() - start_time > duration:
        break
print(
    'The highest max_mean_distance with perfect accuracy after {} rounds of binary search: {}'.format(
        num_iterations,
        best_limit,
    )
)

The highest max_mean_distance with perfect accuracy after 480 rounds of binary search: 0.12336959687424347
